In [ ]:
import pandas as pd

df = pd.read_csv('../data/muscle_atlas/muscle_atlas_2_7_filt_triple_full.csv')
df.head()

In [ ]:
ATP_IMAGE_DF = df[df["Staining method"] == "ATP 9.4"]
ATP_IMAGE_NAME = ATP_IMAGE_DF["Number"].tolist()
IMAGE_INDEX = 189
# show the first image in the list using the image name
from IPython.display import Image
Image(filename='../data/muscle_atlas/images/' + ATP_IMAGE_NAME[IMAGE_INDEX])

In [ ]:
from myoquant.src.common_func import is_gpu_availiable,load_cellpose,run_cellpose
import matplotlib.pyplot as plt
try:
    from imageio.v2 import imread
except ImportError:
    from imageio import imread

image_array = imread('../data/muscle_atlas/images/' + ATP_IMAGE_NAME[IMAGE_INDEX])
model_cellpose = load_cellpose()
mask_cellpose = run_cellpose(image_array, model_cellpose)
plt.imshow(mask_cellpose)

In [ ]:
from skimage.measure import regionprops_table

plt.imshow(mask_cellpose)
props_cellpose = regionprops_table(
    mask_cellpose,
    properties=[
        "label",
        "area",
        "centroid",
        "eccentricity",
        "bbox",
        "image",
        "perimeter",
        "feret_diameter_max",
    ],
)
df_cellpose = pd.DataFrame(props_cellpose)
df_cellpose.head()

In [ ]:
import numpy as np
all_cell_median_intensity = []
for index in range(len(df_cellpose)):
    single_cell_img = image_array[
        df_cellpose.iloc[index, 5] : df_cellpose.iloc[index, 7],
        df_cellpose.iloc[index, 6] : df_cellpose.iloc[index, 8],
    ].copy()

    single_cell_mask = df_cellpose.iloc[index, 9].copy()
    single_cell_img[~single_cell_mask] = 0
    # Calculate median pixel intensity of the cell but ignore 0 values
    single_cell_median_intensity = np.median(single_cell_img[single_cell_img > 0])
    all_cell_median_intensity.append(single_cell_median_intensity)


In [ ]:
from scipy.stats import gaussian_kde

# Build a "density" function based on the dataset
# When you give a value from the X axis to this function, it returns the according value on the Y axis
density = gaussian_kde(all_cell_median_intensity)
density.covariance_factor = lambda : .25
density._compute_covariance()

# Create a vector of 256 values going from 0 to 256:
xs = np.linspace(0, 255, 256)
desnity_xs_values = density(xs)
# Set the figure size
plt.figure(figsize=(8, 4))

# Make the chart
# We're actually building a line chart where x values are set all along the axis and y value are
# the corresponding values from the density function
plt.plot(xs,density(xs))
plt.show()

In [ ]:
# from sklearn.cluster import KMeans
# import numpy as np

# all_cell_median_intensity = np.array(all_cell_median_intensity)

# # fit the k-means model to the data
# kmeans = KMeans(n_clusters=2).fit(all_cell_median_intensity.reshape(-1, 1))

# # get the threshold point between the two clusters
# threshold = kmeans.cluster_centers_[0] if kmeans.cluster_centers_[0] < kmeans.cluster_centers_[1] else kmeans.cluster_centers_[1]


In [ ]:
print(len(density(xs)))
print(len(all_cell_median_intensity))

In [ ]:
from sklearn.mixture import GaussianMixture

# Fit the GMM

gmm = GaussianMixture(n_components=2).fit(np.array(all_cell_median_intensity).reshape(-1, 1))

# Find the x values of the two peaks
peaks_x = gmm.means_.flatten()

print('The x values of the two peaks are:', peaks_x)

In [ ]:
from scipy.stats import norm

sorted_peaks = np.sort(peaks_x)
# Find the minimum point between the two peaks
min_index = np.argmin(density(xs)[(xs > sorted_peaks[0]) & (xs < sorted_peaks[1])])
threshold = sorted_peaks[0]+xs[min_index]
print(threshold)
# Plot the data
plt.plot(xs, density(xs), label='Density')
plt.axvline(threshold, color='r', label='Threshold')
plt.legend()
plt.show()

In [ ]:
df_cellpose["cell_intensity"] = all_cell_median_intensity
df_cellpose["muscle_cell_type"] = df_cellpose["cell_intensity"].apply(
    lambda x: 1 if x > threshold else 2
)
df_cellpose.head()

In [ ]:
df_cellpose["muscle_cell_type"].value_counts(normalize=True)

In [ ]:
label_map = np.zeros(
    (image_array.shape[0], image_array.shape[1]), dtype=np.uint16
)
# for index in track(range(len(df_cellpose)), description="Painting cells"):
for index in range(len(df_cellpose)):
    single_cell_mask = df_cellpose.iloc[index, 9].copy()
    if df_cellpose["muscle_cell_type"][index] == 2:
        label_map[
            df_cellpose.iloc[index, 5] : df_cellpose.iloc[index, 7],
            df_cellpose.iloc[index, 6] : df_cellpose.iloc[index, 8],
        ][single_cell_mask] = 1
    elif df_cellpose["muscle_cell_type"][index] == 1:
        label_map[
            df_cellpose.iloc[index, 5] : df_cellpose.iloc[index, 7],
            df_cellpose.iloc[index, 6] : df_cellpose.iloc[index, 8],
        ][single_cell_mask] = 2

In [ ]:
%config InlineBackend.figure_format = 'retina'
from myoquant.src.common_func import label2rgb, blend_image_with_label
labelRGB_map = label2rgb(image_array, label_map)
overlay_img = blend_image_with_label(image_array, labelRGB_map)

plt.figure(figsize=(10,20))

f, axarr = plt.subplots(1,2) 
axarr[0].imshow(image_array)
axarr[1].imshow(overlay_img)
plt.tight_layout()
plt.show()